## '관리부서', '전화번호', 'X좌표(GRS80TM)', 'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기'
## 1. 공원의 면적을 계산하시오 ( 힌트 : 제일 처음에 있는 데이터 100m2, 정수로 표시)

## 2. 공원의 면적에 따라 분류, 크기
### ~ 100,000 m2 : 소형, 3
### ~ 1,000,000 m2 : 중형, 7
### 1,000,000 m2~ : 대형, 15

## 3. 공원명, 지역, 면적 위도, 경도, 분류, 크기 필드를 갖는 데이터프레임을 만드시오

## 4. 서울시 지도에 공원을 시각화하시오
## - 크기에 따른 CircleMarker
## - tooltip에 '공원명 면적'이 표시되도록


In [1]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 서울시 주요 공원현황.csv to 서울시 주요 공원현황.csv


## 1. 공원의 면적을 계산하시오 ( 힌트 : 제일 처음에 있는 데이터 100m2, 정수로 표시)

In [62]:
import folium
import pandas as pd
import numpy as np
df = pd.read_csv(filename, encoding='EUC-KR')

In [63]:
area = df.면적.astype(str).apply(lambda x: x.strip('총 ').split()[0])
area[-5:]

127    109635㎡
128    140022㎡
129    211392㎡
130    157535㎡
131        nan
Name: 면적, dtype: object

In [64]:
area[(area == 'nan') | (area == '휴양')]

6       휴양
109    nan
119    nan
131    nan
Name: 면적, dtype: object

In [65]:
area[(area == 'nan') | (area == '휴양')] = '0㎡'
area[-5:]

127    109635㎡
128    140022㎡
129    211392㎡
130    157535㎡
131         0㎡
Name: 면적, dtype: object

In [66]:
import re
area = area.apply(lambda x: float(re.sub('[㎡m²㎥]','',x)))

In [67]:
area = area.apply(lambda x: int(np.round(x)))

In [68]:
area

0      2896887
1        80683
2      9132690
3       480994
4      2284085
        ...   
127     109635
128     140022
129     211392
130     157535
131          0
Name: 면적, Length: 132, dtype: int64

In [69]:
df_area = area.to_frame()

In [70]:
df_area

,면적
0,2896887
1,80683
2,9132690
3,480994
4,2284085
...,...
127,109635
128,140022
129,211392
130,157535


## 2. 공원의 면적에 따라 분류, 크기
### ~ 100,000 m2 : 소형, 3
### ~ 1,000,000 m2 : 중형, 7
### 1,000,000 m2~ : 대형, 15

In [71]:
area_criteria = [-1, 100000, 1000000, 12000000]
labels = ['소형', '중형', '대형']
size_info = [3, 7, 15]
scale = pd.cut(area, area_criteria, labels=labels)
size = pd.cut(area, area_criteria, labels=size_info)
scale[:5]

0    대형
1    소형
2    대형
3    중형
4    대형
Name: 면적, dtype: category
Categories (3, object): ['소형' < '중형' < '대형']

In [72]:
size[:5]

0    15
1     3
2    15
3     7
4    15
Name: 면적, dtype: category
Categories (3, int64): [3 < 7 < 15]

## 3. 공원명, 지역, 면적 위도, 경도, 분류, 크기 필드를 갖는 데이터프레임을 만드시오

In [73]:
df = df[['공원명','지역','X좌표(WGS84)','Y좌표(WGS84)']]
df.columns = ['공원명','지역','경도','위도']
df['면적'] = area
df['분류'] = scale
df['크기'] = size
df.head()

,공원명,지역,경도,위도,면적,분류,크기
0,남산도시자연공원,중구,126.990377,37.550140,2896887,대형,15
1,길동생태공원,강동구,127.154779,37.540394,80683,소형,3
2,서울대공원,과천시,127.019847,37.426449,9132690,대형,15
3,서울숲,성동구,127.041798,37.543072,480994,중형,7
4,월드컵공원,마포구,126.878907,37.571805,2284085,대형,15


In [74]:
df.isnull().sum()

공원명    0
지역     1
경도     1
위도     1
면적     0
분류     0
크기     0
dtype: int64

In [75]:
df[df.경도.isna()]

,공원명,지역,경도,위도,면적,분류,크기
131,서울로7017,중구,NaN,NaN,0,소형,3


In [76]:
df = df.dropna()

In [77]:
df.to_csv('서울공원요약.csv', index=False)

## 4. 서울시 지도에 공원을 시각화하시오
## - 크기에 따른 CircleMarker
## - tooltip에 '공원명 면적'이 표시되도록

In [78]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)
for i in df.index:
    folium.CircleMarker([df.위도[i], df.경도[i]], 
                        radius=int(df.크기[i]),
                        tooltip=f"{df.공원명[i]}({df.면적[i]:,d}㎡)",
                        color='crimson', fill_color='crimson').add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>서울시내 공원 위치와 크기</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map

In [79]:

map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
for i in df.index:
    folium.CircleMarker([df.위도[i], df.경도[i]], 
                        radius=int(df.크기[i]),
                        tooltip=f"{df.공원명[i]}({df.면적[i]:,d}㎡)",
                        color='green', fill_color='green').add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>서울시내 공원 위치와 크기</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map